# add Area, SD, HD columns to supersite list
- read in pct_supersite.xlsx
- group by supersite
- extract list of Area, SD, HD


In [21]:
import pandas as pd

In [22]:
# read in pct_supersite.xlsx

cols = ['Pct', 'AreaShort', 'SD', 'HD', 'supersite']
pctss = pd.read_excel('data/pct_supersite.xlsx', usecols=cols)

pctss.info()
pctss

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Pct        193 non-null    int64 
 1   AreaShort  193 non-null    object
 2   SD         193 non-null    int64 
 3   HD         193 non-null    int64 
 4   supersite  193 non-null    object
dtypes: int64(3), object(2)
memory usage: 7.7+ KB


,Pct,AreaShort,SD,HD,supersite
0,2,LM-04,15,19,Trail Ridge MS
1,3,SU-01,18,49,Eldorado K8
2,4,BO-01,15,12,Manhattan MS
3,100,SU-01,18,12,Eldorado K8
4,101,SU-01,18,12,Eldorado K8
...,...,...,...,...,...
188,912,MT-02,15,49,Centennial MS
189,913,MT-02,15,49,Jamestown School
190,914,MT-03,15,49,Allenspark Fire Station
191,915,MT-03,15,49,Lyons Middle Senior


In [23]:
## try create arealist using for loops for Altona MS

pctss[pctss['supersite'] == 'Altona MS']['AreaShort'].unique().tolist()



['LM-02', 'LM-03']

In [24]:
pctss

,Pct,AreaShort,SD,HD,supersite
0,2,LM-04,15,19,Trail Ridge MS
1,3,SU-01,18,49,Eldorado K8
2,4,BO-01,15,12,Manhattan MS
3,100,SU-01,18,12,Eldorado K8
4,101,SU-01,18,12,Eldorado K8
...,...,...,...,...,...
188,912,MT-02,15,49,Centennial MS
189,913,MT-02,15,49,Jamestown School
190,914,MT-03,15,49,Allenspark Fire Station
191,915,MT-03,15,49,Lyons Middle Senior


## Try using groupby
- issues

In [20]:
ssgroup = pctss.groupby('supersite')

ssgroup.ngroups
ssgroup.size()
ssgroup.first()
ssgrp['AreaShort'].unique().tolist()
# ssgroup[['AreaShort']].groups

['MT-03']

In [4]:
# get all the Areas in Altona MS

ssgroup.get_group('Altona MS')['AreaShort'].unique().tolist()

['LM-02', 'LM-03']

In [10]:
# iterate through all supersites

ss_Area = {'areas':[]}
for ss, ssgrp in ssgroup:
    # print(ss)
    # print(type(ssgrp['AreaShort'].unique().tolist()))
    # print(ss,ssgrp['AreaShort'].unique())
    # print({ss:ssgrp['AreaShort'].unique().tolist()})
    ss_Area.append({ss:ssgrp['AreaShort'].unique().tolist()})

ss_Area

AttributeError: 'dict' object has no attribute 'append'

In [9]:

ss_Areadf = pd.DataFrame.from_dict(ss_Area, orient='index')
ss_Areadf

AttributeError: 'list' object has no attribute 'values'

In [46]:
# function to extract unique areas from a group

def col_unique(ssgrp_):
    # for ss, ssvals in ssgrp:
    #     # print(ss)
    #     # print(type(ssvals['AreaShort'].unique().tolist()))
    #     # print(ss,ssvals['AreaShort'].unique())
    #     # print({ss:ssvals['AreaShort'].unique().tolist()})
    #     ss_Area += {ss:ssvals['AreaShort'].unique().tolist()}

    return ssgrp_['AreaShort'].unique().tolist()



[{},
 'Allenspark Fire Station ',
 'Altona MS',
 'Burlington Elementary',
 'Casey MS',
 'Centaurus HS',
 'Centennial MS',
 'Eldorado K8',
 'Erie MS',
 'Gold Hill School',
 'Jamestown School',
 'Longs Peak MS',
 'Lyons Middle Senior',
 'Manhattan MS',
 'Monarch HS',
 'Nederland HS',
 'New Vista',
 'Niwot HS',
 'Southern Hills MS',
 'Trail Ridge MS',
 'Ward Community Center']